In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import csv
import seaborn as sns
import io 

In [3]:
df_old = pd.read_excel(r"C:\Users\eva-k\Documents\Studium\TUM\Master MMT\5. Semester\Cybercrime\data-analytics-bugcrowd\data-octa\bugcrowd_modified_24.12.xlsx")
df_new = pd.read_excel(r"C:\Users\eva-k\Documents\Studium\TUM\Master MMT\5. Semester\Cybercrime\data-analytics-bugcrowd\used_data\07.01withdiscovery.xlsx")


In [4]:
df_new.dtypes

Reward Range                 object
Link                         object
Title                        object
Short Description            object
Is Safe Harbor               object
Vulnearbilities Rewarded    float64
Validation Within            object
Average Payout               object
Hall of Famers               object
Number People                object
P4                           object
P3                           object
P2                           object
P1                           object
Maximum Reword               object
programm_ruels               object
annocument_count            float64
Featured                     object
Industry                     object
Technology                   object
dtype: object

In [5]:
df_new.columns

Index(['Reward Range', 'Link', 'Title', 'Short Description', 'Is Safe Harbor',
       'Vulnearbilities Rewarded', 'Validation Within', 'Average Payout',
       'Hall of Famers', 'Number People', 'P4', 'P3', 'P2', 'P1',
       'Maximum Reword', 'programm_ruels', 'annocument_count', 'Featured',
       'Industry', 'Technology'],
      dtype='object')

In [10]:
#clean reward range
df_new['Reward Range'] = df_new['Reward Range'].str.replace('$', '').str.replace('Points', '0').str.replace(",", "")
df_new['Reward Range'] = df_new['Reward Range'].fillna(0)


0 - 2500


C:\Users\eva-k\AppData\Local\Temp\ipykernel_29600\2001661482.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_new['Reward Range'] = df_new['Reward Range'].str.replace('$', '').str.replace('Points', '0').str.replace(",", "")


In [32]:
#count safe harbor

partial_sh = (df_new['Is Safe Harbor'].value_counts()['Partial safe harbor'])

sh = df_new['Is Safe Harbor'].value_counts()['Safe harbor']
print("SH: ", sh)
print("PSH: ", partial_sh)

l = len(df_new['Is Safe Harbor'])
no_sh= l - sh - partial_sh
print("No: ", no_sh)



SH:  143
PSH:  76
No:  111
Safe harbor            143
NaN                    111
Partial safe harbor     76
Name: Is Safe Harbor, dtype: int64


In [40]:
#clean validation within

# Apply the to_hours() function to the "Validation Within" column
df_new["Validation Within"].apply(lambda x: str(x))
df_new["Validation Within"] = df_new["Validation Within"].fillna(value="N/A")
df_new["Validation Within"].apply(lambda x: str(x))

for i in df_new["Validation Within"]:
    print(i)


N/A
96.0
192.0
N/A
96.0
N/A
72.0
360.0
96.0
N/A
21.0
N/A
456.0
72.0
N/A
72.0
72.0
240.0
N/A
240.0
432.0
N/A
N/A
N/A
96.0
N/A
216.0
N/A
48.0
N/A
72.0
72.0
288.0
72.0
48.0
96.0
72.0
13.0
N/A
72.0
N/A
N/A
120.0
648.0
48.0
168.0
72.0
72.0
96.0
N/A
72.0
360.0
96.0
N/A
N/A
N/A
N/A
96.0
120.0
72.0
72.0
N/A
N/A
N/A
120.0
48.0
1440.0
12.0
96.0
168.0
48.0
72.0
168.0
1440.0
96.0
72.0
N/A
168.0
N/A
96.0
192.0
N/A
168.0
N/A
N/A
N/A
N/A
N/A
48.0
2.0
N/A
N/A
72.0
72.0
48.0
288.0
72.0
7.0
N/A
48.0
N/A
N/A
N/A
96.0
N/A
48.0
N/A
N/A
N/A
192.0
48.0
N/A
N/A
48.0
48.0
72.0
N/A
72.0
48.0
N/A
N/A
96.0
48.0
N/A
48.0
48.0
48.0
192.0
72.0
48.0
N/A
120.0
48.0
144.0
N/A
72.0
48.0
264.0
N/A
96.0
N/A
N/A
96.0
72.0
72.0
72.0
N/A
N/A
120.0
120.0
N/A
48.0
96.0
N/A
72.0
N/A
96.0
13.0
96.0
N/A
144.0
120.0
72.0
N/A
96.0
216.0
672.0
N/A
48.0
N/A
96.0
6.0
48.0
48.0
72.0
72.0
288.0
N/A
N/A
432.0
23.0
13.0
N/A
N/A
72.0
N/A
N/A
N/A
N/A
13.0
48.0
72.0
72.0
48.0
144.0
N/A
48.0
456.0
48.0
72.0
96.0
N/A
48.0
48.0
N/A
48.0
N/A
N/A

In [50]:
import re
from datetime import timedelta

def to_hours(s):
    if pd.isnull(s):
        # If the input is NaN, return NaN
        return pd.np.nan
    elif s == "":
        return pd.np.nan
    else:
        # Extract the numerical value and the unit of time from the string
        m = re.match(r'(\d+) (days|hours|weeks|months)', s)
        if not m:
            # If the input is not in the expected format, check if it starts with "about"
            if s.startswith("about "):
                # If it does, remove the "about" prefix and try again
                s = s[6:]
                m = re.match(r'(\d+) (days|hours|weeks|months)', s)
            
            if not m:
                print(f"Error: Invalid input: {s}")
                return pd.np.nan
        value, unit = m.groups()

        # Convert the value to a float and the unit to lowercase
        value = float(value)
        unit = unit.lower()

        # Convert the value to hours
        if unit == 'days':
            value *= 24
        elif unit == 'hours':
            pass
        elif unit == 'weeks':
            value *= 7 * 24
        elif unit == 'months':
            # Estimate the number of hours in a month based on 30 days
            value *= 30 * 24
        else:
            print(f"Error: Invalid unit: {unit}")
            raise ValueError(f"Invalid unit: {unit}")
    
    return value



df_new['Validation Within'] = df_new['Validation Within'].apply(to_hours, result_type = 'broadcast')


TypeError: to_hours() got an unexpected keyword argument 'result_type'

In [ ]:
for i in df_new["Validation Within"]:
    print(i)

In [36]:
#clean avareage payout 
df_new["Average Payout"] = df_new["Average Payout"].str.replace('$', '').str.replace(",","")
df_new["Average Payout"] = df_new["Average Payout"].fillna("0")
df_new["Average Payout"] = df_new["Average Payout"].astype(float)

C:\Users\eva-k\AppData\Local\Temp\ipykernel_29600\39423583.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_new["Average Payout"] = df_new["Average Payout"].str.replace('$', '').str.replace(",","")


In [37]:
#clean number of people
df_new["Number People"]= df_new["Number People"].str.replace("total","")
df_new["Number People"]= df_new["Number People"].fillna("0")
df_new["Number People"]= df_new["Number People"].astype(int)

In [38]:
#clean Hall of fame
df_new["Hall of Famers"] = df_new["Hall of Famers"].fillna("0")
df_new["Hall of Famers"] = df_new["Hall of Famers"].str.replace("View all ","").str.replace("View the hall","0")
df_new["Hall of Famers"] = df_new["Hall of Famers"].astype(int)

In [39]:
#Clean Priority Payout
def cleanse_priority(row):
    row = row.fillna("0")
    row = row.str.replace("$","")
    return row 

df_new["P1"] = cleanse_priority(df_new["P1"])
df_new["P2"] = cleanse_priority(df_new["P2"])
df_new["P3"] = cleanse_priority(df_new["P3"])
df_new["P4"] = cleanse_priority(df_new["P4"])

C:\Users\eva-k\AppData\Local\Temp\ipykernel_29600\2083858369.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  row = row.str.replace("$","")
